# [Path 路径](https://docs.python.org/3/library/pathlib.html#module-pathlib)

In [27]:
from pathlib import Path

Path(r"C:\Program Files\ ")  # raw-string 反斜杠不被认为是转义符
Path("/usr/local/bin")  # Unix/Linux/Mac
Path("usr/") / Path("Program") / "Hello" # 使用/运算符连接路径
Path.home() # 返回主目录
path = Path.home()
path.exists()  # True
path.is_file() # False
path.is_dir()  # True
print(path.name)  # 如果指向文件，则会返回文件全名
print(path.stem)  # 如果是文件，会除去后缀名后返回
print(path.suffix) # 返回文件后缀
print(path.parent) # 返回父文件夹
print(path.with_name("file.txt"))  # 在path文件目录下创建一个叫file.txt的文件
print(path.absolute())

JackyLee
JackyLee

C:\Users
C:\Users\file.txt
C:\Users\JackyLee


## 对目录进行操作

In [67]:
from pathlib import Path
path = Path("test")
print(path.absolute())
# path.exists()
# path.mkdir()
# path.rmdir()
# path.rename(name)

print(path.iterdir())
for p in path.iterdir():  # 生成器
    print(p)
files = [p for p in path.iterdir() if p.is_dir()]  # 变成列表
print(files)

md_files = [p for p in path.glob("*.md")]  # 使用通配符查找特定后缀文件
md_files = [p for p in path.glob("**/*.md")]  # 使用通配符查找特定后缀文件
print(md_files)


D:\OneDrive\Program\Jupyter\test
<generator object Path.iterdir at 0x0000026CDDE892E0>
test\hello.txt
test\subdir
test\test.md
[WindowsPath('test/subdir')]
[WindowsPath('test/test.md'), WindowsPath('test/subdir/todo.md')]


## 对普通文件进行操作 `00:09:55`

In [100]:
from pathlib import Path
from time import ctime
path = Path("test")
file = Path("test/test.md")
# file.rename("test/init.txt")
# print(file.unlink())  # 如果是普通文件，unlink后就会消失
print(file.stat())
print(file.stat().st_ctime)
print(ctime(file.stat().st_ctime))
# print(path.read_text())  # permisson denied
# print(path.write_test("..."))  # permisson denied
with open("test/test.md", "r") as f:
    print(f.read())
    


os.stat_result(st_mode=33206, st_ino=1125899907370986, st_dev=3799934004, st_nlink=1, st_uid=0, st_gid=0, st_size=3, st_atime=1650641082, st_mtime=1650640819, st_ctime=1650640649)
1650640649.5477602
Fri Apr 22 23:17:29 2022
123


### 拷贝文件

In [99]:
from pathlib import Path
import shutil

source = Path("test/test.md")
target = Path() / "test.md"

# target.write_text(source.read_text())  # 太复杂
shutil.copy(source, target)

WindowsPath('test.md')

## 对压缩文件进行操作

In [122]:
from pathlib import Path
from zipfile import ZipFile

with ZipFile("files.zip", "w") as zip:
    for path in Path("test").rglob("*.*"):
        zip.write(path)
with ZipFile("files.zip", "r") as zip:
    print(zip.namelist())
    info = zip.getinfo("test/test.md")
    print(info.file_size)
    print(info.compress_size)
    zip.extractall("extract")

['test/hello.txt', 'test/test.md', 'test/subdir/todo.md']
3
3


## 对CSV文件进行操作

In [133]:
from pathlib import Path
import csv
# 写操作
with open("data.csv", "w", newline='') as file:  # windows需要加上newline=''
    writer = csv.writer(file)
    writer.writerow(["交易号", "产品号", "价格"])
    writer.writerow([1000, 1, 5])
    writer.writerow([1001, 2, 6])
# 读操作
with open("data.csv") as file: 
    reader = csv.reader(file)
#     print(list(reader))  # 文件头会走到最后
    for row in reader:
        print(row)


['交易号', '产品号', '价格']
['1000', '1', '5']
['1001', '2', '6']


## 对JSON文件进行操作

In [142]:
from pathlib import Path
import json
movies = [
    {"id":1, "title": "Terminator", "year": 1989},
    {"id":2, "title": "Lion", "year": 1997}
]

data = json.dumps(movies)

with open("moives.json", "wt") as f:
    f.write(data)
with open("moives.json", "rt") as f:
    data = f.read()
    print(data)
    movies = json.loads(data)
    print(movies[0])

[{"id": 1, "title": "Terminator", "year": 1989}, {"id": 2, "title": "Lion", "year": 1997}]
{'id': 1, 'title': 'Terminator', 'year': 1989}


## 对数据库SQLite处理

[sqlite数据库下载](https://sqlitebrowser.org/dl/)

In [153]:
import sqlite3
import json
from pathlib import Path

# 读入json数据
movies = []
with open("moives.json", "rt") as f:
    movies = json.loads(f.read())
print(movies)

with sqlite3.connect("db.sqlite3") as conn:
    command = "INSERT INTO Movies VALUES(?, ?, ?)"
    # 创建数据库的部分在视频30min左右
    print(movies[0].values())
    print(tuple(movies[0].values()))
    for movie in movies:
        conn.execute(command,tuple(movie.values()))
    conn.commit()

[{'id': 1, 'title': 'Terminator', 'year': 1989}, {'id': 2, 'title': 'Lion', 'year': 1997}]
dict_values([1, 'Terminator', 1989])
(1, 'Terminator', 1989)


### 从Sqlite读取数据

In [157]:
import sqlite3
import json
from pathlib import Path

with sqlite3.connect("db.sqlite3") as conn:
    command = "SELECT * FROM Movies"
    cursor = conn.execute(command)
#     for row in cursor:
#         print(row)
    movies = cursor.fetchall() # 和上面等价，返回的是一个列表
    print(movies)

[(1, 'Terminator', 1989), (2, 'Lion', 1997)]


## 处理时间戳

In [159]:
import time

def send_emails():
    for i in range(10000):
        pass
    
start = time.time()
send_emails()
end = time.time()
duration = end - start
print(duration)

0.0010004043579101562


## 处理普通时间格式

关于strptime的格式，可以看[官网](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes)

In [173]:
from datetime import datetime
import time

dt = datetime(2018, 1, 1)
dt = datetime.now()
dt = datetime.strptime("2020/01/02", "%Y/%m/%d")
dt = datetime.fromtimestamp(time.time())
print(dt)
print(f"{dt.year}/{dt.month}")
print(dt.strftime("%Y/%m"))
print()
      

2022-04-23 01:48:51.417108
2022/4
2022/04


## 处理 Δt （时间差）

In [180]:
from datetime import datetime, timedelta

dt1 = datetime(2022, 1, 1) 
dt2 = datetime.now()
duration = dt2 - dt1
print(duration)
print("days", duration.days)
print("seconds", duration.seconds)
print("total_seconds", duration.total_seconds())

dt3 = datetime(2022, 2, 2) + timedelta(1)
dt4 = datetime(2022, 2, 2) + timedelta(days=1,seconds=600)
print(dt4 - dt3)

112 days, 13:20:16.137484
days 112
seconds 48016
total_seconds 9724816.137484
0:10:00


## 生成随机数

In [208]:
import random

random.random() # 生成一个0~1的随机数
random.randint(1,10)  # 生成一个1~10的随机数，左闭右闭
random.choice([1,2,3,4])  # 从给定范围内选择1次
random.choices([1,2,3,4], k=5)  # 从给定范围内选择k次
random.choices("abcdefghi", k=4) # 从给定字符串中选择k次
"".join(random.choices("abcdefghi", k=4)) # 从给定字符串中选择k次，并返回一个字符串

import string
random.choices(string.ascii_letters, k=4) # 从ascii字符中选择或者（string.ascii_lowercase、string.ascii_uppercase）
numbers = [1,2,3,'a']
random.shuffle(numbers)  # 对列表中的元素随机排序（洗牌）
print(numbers)

['a', 2, 3, 1]


## 浏览器 `49:30`

In [210]:
import webbrowser
webbrowser.open("http://baidu.com")

True

## 写邮件 `54:23`

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

message = MIMEMultipart()
message["from"] = "Jackylee"
message["to"] = "779557618@qq.com"
message["subject"] = "this is a test"
message.attach(MIMEText("Body"))
with smtplib.SMTP(host="smtp.gmail.com", port=587) as smtp:
    smtp.ehlo()
    smtp.starttls()
    smtp.login("xxx@xxx.com", "xxx")
    smtp.send_message(message)
    print("send...")

## 模板 `57:32`

In [217]:
from string import Template

## 命令行 `1:02:25`

In [218]:
import sys
print(sys.argv)

['C:\\Users\\JackyLee\\AppData\\Roaming\\Python\\Python39\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\JackyLee\\AppData\\Roaming\\jupyter\\runtime\\kernel-50ce709e-963e-4382-95d4-84f8cd015714.json']


## 运行外部程序 `1:05:42`

In [221]:
import subprocess
completed = subprocess.run(["dir", ""])  # linux ["ls", "-l"]

FileNotFoundError: [WinError 2] 系统找不到指定的文件。